# YouTube
Tem como objetivo baixar videos no YouTube.

## Setup

In [ ]:
from pytube import YouTube as PyTube, StreamQuery
from pathlib import Path
import pandas as pd
from IPython.display import clear_output
from functools import cache

YOUTUBE_PATH = Path("./files/youtube")
YOUTUBE_PATH.mkdir(exist_ok=True)


class YouTube:
    def __init__(self, url: str):
        self._yt = PyTube(url=url, on_progress_callback=self._progress)
        self.url = url
        self.autor = self._yt.author
        self.titulo = self._yt.title
        self._filesize: int = 0

    def __str__(self) -> str:
        return f"<Youtube autor='{self.autor}' titulo='{self.titulo}'>"

    def __repr__(self) -> str:
        return str(self)

    @property
    def _streams(self) -> StreamQuery:
        @cache
        def wrapper():
            return self._yt.streams
        return wrapper()

    def _progress(self, chunk, file_handle: bytes, bytes_remaining: int):
        percent = ((self._filesize - bytes_remaining) * 100) / self._filesize
        block = int((percent * 20) / 100)
        clear_output(wait=True)
        print("Progresso: [{0}] {1:.1f}%".format("#" * block + "-" * (20 - block), percent))

    def resolucoes(self, only_progressive: bool = False) -> pd.DataFrame:
        resolutions = self._streams
        if only_progressive:
            resolutions = resolutions.filter(progressive=True)
        df = pd.DataFrame([res.__dict__ for res in resolutions])
        return df
    
    def baixar_video(self, itag: int) -> str:
        video = self._streams.get_by_itag(itag)
        self._filesize = video.filesize
        return video.download(output_path=str(YOUTUBE_PATH))


## Execução
Executar os 3 passos abaixo na sequencia:

### 1. Digitar URL do video

In [ ]:
youtube_url = "https://www.youtube.com/watch?v=RPf8cYrA9wQ"
yt = YouTube(url=youtube_url)
yt

### 2. Listar Resoluções

In [ ]:
# Somente campos
", ".join(list(yt.resolucoes()))

In [ ]:
yt.resolucoes(only_progressive=True)[["itag", "mime_type", "video_codec", "audio_codec", "fps", "resolution", "abr"]]

### 3. Baixar Video

In [ ]:
yt.baixar_video(itag=18)